# This contains many other modle i trained i inlcuded their test result as given by kaggle 
### many of them may look out of place and there is repitetion of code as they belong to different files which i combined together 
### i also tried xception model but my it was taking too much time so i decided to drop it 

In [ ]:
# vgg16 model accuracy - 87 
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
import pandas as pd
img_width, img_height = 224, 224  
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Data/Train'
validation_data_dir = 'Data/Validation'
nb_train_samples = 801
nb_validation_samples = 325
epochs = 50
batch_size = 16
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)
    model = applications.VGG16(include_top=False, weights='imagenet')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),  
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict(generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy', bottleneck_features_train) 
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),  
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict(generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy', bottleneck_features_validation)  
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')  
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))
    if len(train_labels) < nb_train_samples:
        train_labels = np.append(train_labels, 0)  
    validation_data = np.load('bottleneck_features_validation.npy')  
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))
    if len(validation_labels) < nb_validation_samples:
        validation_labels = np.append(validation_labels, 0) 
    print("Train data shape:", train_data.shape)
    print("Train labels shape:", train_labels.shape)
    print("Validation data shape:", validation_data.shape)
    print("Validation labels shape:", validation_labels.shape)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))  
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    test_dir = '/Users/aniket/cynaptics/Data/Test'
    image_filenames = sorted(os.listdir(test_dir), key=lambda x: int(x.split('_')[1].split('.')[0]))
    predictions = []
    for filename in image_filenames:
        img_path = os.path.join(test_dir, filename)
        try:
            img = load_img(img_path, target_size=(img_width, img_height))  
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0) / 255.0
            pred = model.predict(img_array)
            predicted_class = 'Real' if pred[0] >= 0.5 else 'AI'
            filename_without_extension = os.path.splitext(filename)[0]
            predictions.append((filename_without_extension, predicted_class))
        except Exception as e:
            print(f"Error processing image {filename}: {e}")
    results_df = pd.DataFrame(predictions, columns=['Id', 'Label'])
    results_df.to_csv('predictions.csv', index=False)
save_bottleneck_features()
train_top_model()


In [ ]:
# note define val  and train dir for val i randomly selected 50 images from each class 

import efficientnet.keras as efn
train_datagen4 = ImageDataGenerator(
    rescale=1. / 255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen4 = ImageDataGenerator(
    rescale=1.0 / 255.
)

train_generator4 = train_datagen4.flow_from_directory(
    train_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(224, 224)
)

validation_generator4 = test_datagen4.flow_from_directory(
    val_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(224, 224)
)
base_model = efn.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
for layer in base_model.layers:
    layer.trainable = False

from tensorflow.keras.layers import Dropout, Flatten, Dense 
from tensorflow.keras.models import Model  
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras import optimizers

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)  
predictions = Dense(1, activation=tf.nn.sigmoid)(x)  

model_final = Model(inputs=base_model.input, outputs=predictions)

model_final.compile(
    optimizer=optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

eff_history = model_final.fit(
    train_generator4,
    validation_data=validation_generator4,
    steps_per_epoch=10,
    epochs=15,
    callbacks=[early_stopping]
)



In [ ]:
# base code inception model rest code was same  add test and val  directory 
from tensorflow.keras.applications.inception_v3 import InceptionV3
train_datagen3 = ImageDataGenerator(
    rescale=1. / 255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255.
)
train_generator3 = train_datagen3.flow_from_directory(
    train_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150, 150)
)

validation_generator3 = test_datagen.flow_from_directory(
    val_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150, 150)
)


base_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)
for layer in base_model.layers:
    layer.trainable = False
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model2 = tf.keras.models.Model(base_model.input, x)

model2.compile(optimizer = RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
inc_history = model2.fit(
    train_generator3,
    validation_data=validation_generator3,
    epochs=15,
    callbacks=[early_stopping]
)


In [ ]:
# thsi code help to create val data and check if it already exist 
import os
import random
import shutil

train_dir = 'your path '  
val_dir = 'your path '  

# Create validation directory structure
os.makedirs(os.path.join(val_dir, 'AI'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Real'), exist_ok=True)

# Function to copy random images
def copy_random_images(src_class_dir, dst_class_dir, num_images):
    images = os.listdir(src_class_dir)
    selected_images = random.sample(images, min(num_images, len(images)))
    
    for image in selected_images:
        shutil.copy(os.path.join(src_class_dir, image), os.path.join(dst_class_dir, image))

# Copy images from AI class
copy_random_images(os.path.join(train_dir, 'AI'), os.path.join(val_dir, 'AI'), 50)

# Copy images from Real class
copy_random_images(os.path.join(train_dir, 'Real'), os.path.join(val_dir, 'Real'), 50)

print("Validation images copied successfully.")


In [ ]:
# a differet train data generator thatr i applied earlier  but it turned out they decreasce the test performance on kaggle 
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') 
